# Launch post-run analysis jobs
Code to create a batch script for launching jobs on cori
Sep 1, 2020

In [1]:
import os
import glob
import time
import subprocess as sp
import numpy as np


In [2]:
curr_dir=os.getcwd()
print(curr_dir)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN


In [3]:
val_files={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/',
          '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'
}

In [4]:
results_loc='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'

In [5]:
ls /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/

20210113_072908_cgan_predict_1.1_m2/   20210127_213646_cgan_predict_0.5_m2/
20210113_092234_cgan_predict_0.65_m2/  20210204_130113_cgan_predict_1.1_m2/
20210113_100739_cgan_predict_0.5_m2/   20210204_143542_cgan_predict_0.65_m2/
20210113_185116_cgan_predict_0.65_m2/  20210205_062936_cgan_predict_0.65_m2/
20210119_174849_cgan_predict_0.65_m2/  20210205_063709_cgan_predict_1.1_m2/
20210122_074619_cgan_predict_1.1_m2/   20210205_120608_cgan_predict_0.5_m2/
20210122_095539_cgan_predict_0.5_m2/   20210205_122504_cgan_predict_0.8_m2/
20210127_201829_lambda2.0/             simple_gan_best_runs/


In [6]:
# glob.glob(results_loc+'*')

In [33]:
ip_folder='20210205_122504_cgan_predict_0.8_m2'

In [34]:
### Default dictionary
img_size=128
dict_pars={'job_name':'analysis_128', 'queue':'debug','bins':'uneven','cores':'64',
           'time':'00:30:00'}

dict_pars['val_file']=val_files[str(img_size)]
dict_pars['ip_folder']=results_loc+ip_folder

print(dict_pars)

{'job_name': 'analysis_128', 'queue': 'debug', 'bins': 'uneven', 'cores': '64', 'time': '00:30:00', 'val_file': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/', 'ip_folder': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210205_122504_cgan_predict_0.8_m2'}


In [35]:
assert os.path.isdir(dict_pars['ip_folder'])

In [36]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos={queue}
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time={time}
#SBATCH --job-name={job_name}

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

conda activate v3
code_dir='/global/homes/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN/1_main_code'

python $code_dir/post_analysis_pandas.py -f {ip_folder}  -v {val_file} -c {cores} -bin {bins}
conda deactivate
echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [37]:
# bash_strg

In [38]:
### Write bash file
staging_loc='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/'
fname='batch_analysis.sh'
filename=staging_loc+fname
print(filename)
with open (filename,'w') as f:
    f.write(bash_strg)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/batch_analysis.sh


In [39]:
### Move to staging locations in project space:
os.chdir(staging_loc)

In [40]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=debug
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time=00:30:00
#SBATCH --job-name=analysis_128

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

conda activate v3
code_dir='/global/homes/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN/1_main_code'

python $code_dir/post_analysis_pandas.py -f /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210205_122504_cgan_predict_0.8_m2  -v /global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/ -c 64 -bin uneven
conda deactivate
echo "--end date" `date` `date +%s`


### Submit jobs to cori queue

In [41]:
%%bash -s "$filename" ## Use python variable in bash
sbatch $1

Submitted batch job 38889963
